## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-20-15-45-00 +0000,wsj,Home sales rose to an eight-month high in Octo...,https://www.wsj.com/economy/housing/home-sales...
1,2025-11-20-15-42-26 +0000,nyt,"The Climate Is Changing, Cows Are Stressed and...",https://www.nytimes.com/2025/11/19/world/europ...
2,2025-11-20-15-39-12 +0000,nyt,Ukraine and Europe Chafe at Being Excluded Fro...,https://www.nytimes.com/2025/11/20/world/europ...
3,2025-11-20-15-36-54 +0000,bbc,Trans people could be barred from services bas...,https://www.bbc.com/news/articles/cy0y07959djo...
4,2025-11-20-15-36-45 +0000,bbc,Nigerian separatist leader sentenced to life f...,https://www.bbc.com/news/articles/cj3z38e6v45o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
41,trump,39
374,house,20
120,new,19
175,epstein,16
605,ai,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
214,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...,123
226,2025-11-20-01-59-00 +0000,wsj,President Trump signed legislation authorizing...,https://www.wsj.com/politics/policy/trump-sign...,106
215,2025-11-20-02-32-00 +0000,wsj,Trump Says He Will Meet With Mamdani Friday at...,https://www.wsj.com/politics/elections/trump-s...,104
229,2025-11-20-01-41-44 +0000,wapo,U.S. pushes new Ukraine peace plan as Trump’s ...,https://www.washingtonpost.com/national-securi...,101
238,2025-11-20-01-00-00 +0000,wsj,By feting Saudi Crown Prince Mohammed bin Salm...,https://www.wsj.com/politics/policy/trump-puts...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
214,123,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...
201,89,2025-11-20-03-35-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
292,73,2025-11-19-21-43-13 +0000,nypost,"Elon Musk, Nvidia unveil deal with Saudi state...",https://nypost.com/2025/11/19/business/elon-mu...
78,52,2025-11-20-12-36-44 +0000,cbc,"Europe, Ukraine must be involved in U.S.-Russi...",https://www.cbc.ca/news/world/us-russia-ukrain...
378,49,2025-11-19-16-32-56 +0000,nypost,"Trump Media stock crashes to all-time lows, wi...",https://nypost.com/2025/11/19/business/trump-m...
192,47,2025-11-20-04-23-28 +0000,nypost,Ex-GOP aide paid fetish artist to mutilate her...,https://nypost.com/2025/11/19/us-news/ex-gop-a...
10,46,2025-11-20-15-24-03 +0000,startribune,"U.S. employers added surprisingly solid 119,00...",https://www.startribune.com/late-arriving-sept...
352,42,2025-11-19-18-28-15 +0000,cbc,Israeli strikes kill at least 25 Palestinians ...,https://www.cbc.ca/news/world/israel-gaza-ceas...
238,42,2025-11-20-01-00-00 +0000,wsj,By feting Saudi Crown Prince Mohammed bin Salm...,https://www.wsj.com/politics/policy/trump-puts...
185,34,2025-11-20-05-45-10 +0000,nypost,"Ex-Conn. school employee texted boy, 11, nearl...",https://nypost.com/2025/11/20/us-news/connecti...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
